In [3]:
import pandas as pd
import pandas_market_calendars as mcal


dates = pd.read_excel('earning dates.xlsx')
#dates.set_index('Ticker', inplace=True)
tickers = pd.read_csv('tickers2018_2023.csv')



In [4]:
dates

,Name,Ticker,Date,Event Type
0,Commercial Metals Co,CMC US,2018-01-03 06:45:00,ER
1,RPM International Inc,RPM US,2018-01-04 06:45:00,ER
2,Walgreens Boots Alliance Inc,WBA US,2018-01-04 07:00:00,ER
3,Lamb Weston Holdings Inc,LW US,2018-01-04 08:30:00,ER
4,Aehr Test Systems,AEHR US,2018-01-04 16:06:00,ER
...,...,...,...,...
20126,Universal Health Services Inc,UHS US,2024-04-25 07:00:00,ER
20127,Alphabet Inc,GOOGL US,2024-04-25 07:00:00,ER
20128,Hawaiian Holdings Inc,HA US,2024-04-25 07:00:00,ER
20129,Enphase Energy Inc,ENPH US,2024-04-25 07:00:00,ER


In [5]:
tickers

,ticker
0,TROW
1,ADX
2,HON
3,RMCF
4,AMRC
...,...
7155,VPG
7156,FN
7157,SANW
7158,TSLA


In [6]:
dates['Short_ticker'] = dates['Ticker'].apply(lambda x: x.split()[0] if x.split()[0] != 'US' else x.split()[1] if len(x.split()) > 1 else '')
dates


,Name,Ticker,Date,Event Type,Short_ticker
0,Commercial Metals Co,CMC US,2018-01-03 06:45:00,ER,CMC
1,RPM International Inc,RPM US,2018-01-04 06:45:00,ER,RPM
2,Walgreens Boots Alliance Inc,WBA US,2018-01-04 07:00:00,ER,WBA
3,Lamb Weston Holdings Inc,LW US,2018-01-04 08:30:00,ER,LW
4,Aehr Test Systems,AEHR US,2018-01-04 16:06:00,ER,AEHR
...,...,...,...,...,...
20126,Universal Health Services Inc,UHS US,2024-04-25 07:00:00,ER,UHS
20127,Alphabet Inc,GOOGL US,2024-04-25 07:00:00,ER,GOOGL
20128,Hawaiian Holdings Inc,HA US,2024-04-25 07:00:00,ER,HA
20129,Enphase Energy Inc,ENPH US,2024-04-25 07:00:00,ER,ENPH


In [7]:
earnings_dates_for_tickers = dates[dates['Short_ticker'].isin(tickers['ticker'])]

earnings_dates_for_tickers = earnings_dates_for_tickers.sort_values(by=['Ticker', 'Date'])

earnings_dates_for_tickers


,Name,Ticker,Date,Event Type,Short_ticker
396,Agilent Technologies Inc,A US,2018-02-14 16:05:00,ER,A
1298,Agilent Technologies Inc,A US,2018-05-14 16:05:00,ER,A
2023,Agilent Technologies Inc,A US,2018-08-14 16:05:00,ER,A
2754,Agilent Technologies Inc,A US,2018-11-19 16:05:00,ER,A
3297,Agilent Technologies Inc,A US,2019-02-20 16:05:00,ER,A
...,...,...,...,...,...
15969,Zoetis Inc,ZTS US,2023-02-14 07:00:00,ER,ZTS
16912,Zoetis Inc,ZTS US,2023-05-04 07:00:00,ER,ZTS
17936,Zoetis Inc,ZTS US,2023-08-08 07:00:00,ER,ZTS
18652,Zoetis Inc,ZTS US,2023-11-02 07:00:00,ER,ZTS


In [8]:
columns_ordered = ['Short_ticker'] + [col for col in earnings_dates_for_tickers.columns if col != 'Short_ticker']
earnings_dates_for_tickers = earnings_dates_for_tickers[columns_ordered]
earnings_dates_for_tickers


,Short_ticker,Name,Ticker,Date,Event Type
396,A,Agilent Technologies Inc,A US,2018-02-14 16:05:00,ER
1298,A,Agilent Technologies Inc,A US,2018-05-14 16:05:00,ER
2023,A,Agilent Technologies Inc,A US,2018-08-14 16:05:00,ER
2754,A,Agilent Technologies Inc,A US,2018-11-19 16:05:00,ER
3297,A,Agilent Technologies Inc,A US,2019-02-20 16:05:00,ER
...,...,...,...,...,...
15969,ZTS,Zoetis Inc,ZTS US,2023-02-14 07:00:00,ER
16912,ZTS,Zoetis Inc,ZTS US,2023-05-04 07:00:00,ER
17936,ZTS,Zoetis Inc,ZTS US,2023-08-08 07:00:00,ER
18652,ZTS,Zoetis Inc,ZTS US,2023-11-02 07:00:00,ER


In [9]:
# Create a market calendar for NYSE
nyse = mcal.get_calendar('NYSE')

def adjust_trading_date(row, calendar):
    date = row['Date'].date()
    time = row['Date'].time()
    
    if time < pd.Timestamp('16:00').time():
        # Get the previous trading day
        valid_days = calendar.valid_days(start_date=date - pd.Timedelta(days=10), end_date=date)
        previous_trading_day = valid_days[-2]  # The last one is the current day, so we take the one before that
        return previous_trading_day
    else:
        # Get the next trading day
        valid_days = calendar.valid_days(start_date=date, end_date=date + pd.Timedelta(days=10))
        next_trading_day = valid_days[1]  # The first one is the current day, so we take the next one
        return next_trading_day

earnings_dates_for_tickers['Adjusted Trading Date'] = earnings_dates_for_tickers.apply(lambda row: adjust_trading_date(row, nyse), axis=1)


earnings_dates_for_tickers



C:\Users\swa19\AppData\Local\Temp\ipykernel_28464\3948966864.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  earnings_dates_for_tickers['Adjusted Trading Date'] = earnings_dates_for_tickers.apply(lambda row: adjust_trading_date(row, nyse), axis=1)


,Short_ticker,Name,Ticker,Date,Event Type,Adjusted Trading Date
396,A,Agilent Technologies Inc,A US,2018-02-14 16:05:00,ER,2018-02-15 00:00:00+00:00
1298,A,Agilent Technologies Inc,A US,2018-05-14 16:05:00,ER,2018-05-15 00:00:00+00:00
2023,A,Agilent Technologies Inc,A US,2018-08-14 16:05:00,ER,2018-08-15 00:00:00+00:00
2754,A,Agilent Technologies Inc,A US,2018-11-19 16:05:00,ER,2018-11-20 00:00:00+00:00
3297,A,Agilent Technologies Inc,A US,2019-02-20 16:05:00,ER,2019-02-21 00:00:00+00:00
...,...,...,...,...,...,...
15969,ZTS,Zoetis Inc,ZTS US,2023-02-14 07:00:00,ER,2023-02-13 00:00:00+00:00
16912,ZTS,Zoetis Inc,ZTS US,2023-05-04 07:00:00,ER,2023-05-03 00:00:00+00:00
17936,ZTS,Zoetis Inc,ZTS US,2023-08-08 07:00:00,ER,2023-08-07 00:00:00+00:00
18652,ZTS,Zoetis Inc,ZTS US,2023-11-02 07:00:00,ER,2023-11-01 00:00:00+00:00


**Statistcis of how many tickers do we have**

In [10]:
earnings_dates_for_tickers['Short_ticker'].unique().size

848

In [11]:
count_before_2024 = earnings_dates_for_tickers[earnings_dates_for_tickers['Date'] < pd.Timestamp(year=2024, month=1, day=1)].shape[0]

print(f"There are {count_before_2024} rows with dates before 2024.")


There are 18468 rows with dates before 2024.


earnings_dates_for_tickers.to_parquet('tickers_ED_20182023.parquet', index=False)

In [45]:
earnings_dates_for_tickers=earnings_dates_for_tickers.reset_index()

In [46]:
earnings_dates_for_tickers

,index,Short_ticker,Name,Ticker,Date,Event Type,Adjusted Trading Date
0,396,A,Agilent Technologies Inc,A US,2018-02-14 16:05:00,ER,2018-02-15 00:00:00+00:00
1,1298,A,Agilent Technologies Inc,A US,2018-05-14 16:05:00,ER,2018-05-15 00:00:00+00:00
2,2023,A,Agilent Technologies Inc,A US,2018-08-14 16:05:00,ER,2018-08-15 00:00:00+00:00
3,2754,A,Agilent Technologies Inc,A US,2018-11-19 16:05:00,ER,2018-11-20 00:00:00+00:00
4,3297,A,Agilent Technologies Inc,A US,2019-02-20 16:05:00,ER,2019-02-21 00:00:00+00:00
...,...,...,...,...,...,...,...
19471,15969,ZTS,Zoetis Inc,ZTS US,2023-02-14 07:00:00,ER,2023-02-13 00:00:00+00:00
19472,16912,ZTS,Zoetis Inc,ZTS US,2023-05-04 07:00:00,ER,2023-05-03 00:00:00+00:00
19473,17936,ZTS,Zoetis Inc,ZTS US,2023-08-08 07:00:00,ER,2023-08-07 00:00:00+00:00
19474,18652,ZTS,Zoetis Inc,ZTS US,2023-11-02 07:00:00,ER,2023-11-01 00:00:00+00:00


In [127]:
import pandas_market_calendars
from pandas import Timestamp
from datetime import datetime, timedelta
ls=list(nyse.valid_days(start_date="2015-01-01", end_date="2024-12-31"))
ls=[ts.tz_convert(None).date()  for ts in ls]
def find_entry_trade_date(timestamp):
    [date_to_check,time]=timestamp.strftime('%Y-%m-%d %H:%M:%S').split(' ')
    #date_to_check = datetime.strptime(date_to_check, '%Y-%m-%d')
    if int(time[0:2]) >16:
        if timestamp.to_pydatetime().date() in ls:
            return(timestamp)
        else:
            invalid=True
            while invalid:
                timestamp= timestamp - timedelta(days=1)
                if timestamp.to_pydatetime().date() in ls:
                    return(timestamp)
            
    else:
        invalid=True
        while invalid:
            timestamp= timestamp - timedelta(days=1)
            if timestamp.to_pydatetime().date() in ls:
                return(timestamp)

def find_exit_trade_date(timestamp):
    invalid=True
    while invalid:
        timestamp= timestamp + timedelta(days=1)
        if timestamp.to_pydatetime().date() in ls:
            return(timestamp)
    

In [128]:
earnings_dates_for_tickers

,index,Short_ticker,Name,Ticker,Date,Event Type,Adjusted Trading Date,trade_entry
0,396,A,Agilent Technologies Inc,A US,2018-02-14 16:05:00,ER,2018-02-15 00:00:00+00:00,2018-02-13 16:05:00
1,1298,A,Agilent Technologies Inc,A US,2018-05-14 16:05:00,ER,2018-05-15 00:00:00+00:00,2018-05-11 16:05:00
2,2023,A,Agilent Technologies Inc,A US,2018-08-14 16:05:00,ER,2018-08-15 00:00:00+00:00,2018-08-13 16:05:00
3,2754,A,Agilent Technologies Inc,A US,2018-11-19 16:05:00,ER,2018-11-20 00:00:00+00:00,2018-11-16 16:05:00
4,3297,A,Agilent Technologies Inc,A US,2019-02-20 16:05:00,ER,2019-02-21 00:00:00+00:00,2019-02-19 16:05:00
...,...,...,...,...,...,...,...,...
19471,15969,ZTS,Zoetis Inc,ZTS US,2023-02-14 07:00:00,ER,2023-02-13 00:00:00+00:00,2023-02-13 07:00:00
19472,16912,ZTS,Zoetis Inc,ZTS US,2023-05-04 07:00:00,ER,2023-05-03 00:00:00+00:00,2023-05-03 07:00:00
19473,17936,ZTS,Zoetis Inc,ZTS US,2023-08-08 07:00:00,ER,2023-08-07 00:00:00+00:00,2023-08-07 07:00:00
19474,18652,ZTS,Zoetis Inc,ZTS US,2023-11-02 07:00:00,ER,2023-11-01 00:00:00+00:00,2023-11-01 07:00:00


In [129]:
earnings_dates_for_tickers['trade_entry']=earnings_dates_for_tickers['Date'].apply(lambda x: find_entry_trade_date(x))
earnings_dates_for_tickers['trade_exit']=earnings_dates_for_tickers['trade_entry'].apply(lambda x: find_exit_trade_date(x))

In [130]:
earnings_dates_for_tickers

,index,Short_ticker,Name,Ticker,Date,Event Type,Adjusted Trading Date,trade_entry,trade_exit
0,396,A,Agilent Technologies Inc,A US,2018-02-14 16:05:00,ER,2018-02-15 00:00:00+00:00,2018-02-13 16:05:00,2018-02-14 16:05:00
1,1298,A,Agilent Technologies Inc,A US,2018-05-14 16:05:00,ER,2018-05-15 00:00:00+00:00,2018-05-11 16:05:00,2018-05-14 16:05:00
2,2023,A,Agilent Technologies Inc,A US,2018-08-14 16:05:00,ER,2018-08-15 00:00:00+00:00,2018-08-13 16:05:00,2018-08-14 16:05:00
3,2754,A,Agilent Technologies Inc,A US,2018-11-19 16:05:00,ER,2018-11-20 00:00:00+00:00,2018-11-16 16:05:00,2018-11-19 16:05:00
4,3297,A,Agilent Technologies Inc,A US,2019-02-20 16:05:00,ER,2019-02-21 00:00:00+00:00,2019-02-19 16:05:00,2019-02-20 16:05:00
...,...,...,...,...,...,...,...,...,...
19471,15969,ZTS,Zoetis Inc,ZTS US,2023-02-14 07:00:00,ER,2023-02-13 00:00:00+00:00,2023-02-13 07:00:00,2023-02-14 07:00:00
19472,16912,ZTS,Zoetis Inc,ZTS US,2023-05-04 07:00:00,ER,2023-05-03 00:00:00+00:00,2023-05-03 07:00:00,2023-05-04 07:00:00
19473,17936,ZTS,Zoetis Inc,ZTS US,2023-08-08 07:00:00,ER,2023-08-07 00:00:00+00:00,2023-08-07 07:00:00,2023-08-08 07:00:00
19474,18652,ZTS,Zoetis Inc,ZTS US,2023-11-02 07:00:00,ER,2023-11-01 00:00:00+00:00,2023-11-01 07:00:00,2023-11-02 07:00:00
